In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import scipy.constants
from scipy.signal import savgol_filter
import hvplot.pandas

%run "C:\Users\conde\Desktop\My Archives\Codes\Python\Data Analysis\my_optical_analysis_functions.ipynb"

In [ ]:
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\LTOI e LN on Sapphire\\LTOI"
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\On-Chip Backward Stimulated Brillouin Scattering in Lithium Niobate Waveguides\\Data\\Fig10_Supp"

directory_file_list = os.listdir(directory)
selected_files = []
counter = 0

for i in directory_file_list:
    if i.endswith(".csv") or i.endswith(".parq"):
        print(f"{counter}: {i}")
        selected_files.append(i)
        counter += 1

In [ ]:
idx_file = 0
header = 21

file_name = selected_files[idx_file]
#df_raw = pd.read_csv(directory+"\\"+selected_files[idx_file], header=header)
df_raw = pd.read_parquet(directory+"\\"+selected_files[idx_file])
columns_raw=df_raw.columns

print(file_name)
df_raw.hvplot.explorer(cmap='viridis', rasterize=True)

In [4]:
idx_crop_i = int(4.417e5)
idx_crop_f = int(7.4e5)
idx_data_column = 2
idx_MZI_column = 1
idx_HCN_column = 2
flip = True
#----------

data_crop = (df_raw[columns_raw[idx_data_column]].values)[idx_crop_i:idx_crop_f]
MZI_crop = (df_raw[columns_raw[idx_MZI_column]].values)[idx_crop_i:idx_crop_f]
HCN_crop = (df_raw[columns_raw[idx_HCN_column]].values)[idx_crop_i:idx_crop_f]

if flip:
    data_crop = np.flip(data_crop)
    MZI_crop = np.flip(MZI_crop)
    HCN_crop = np.flip(HCN_crop)

In [ ]:
MZI_normalized, MZI_baseline_up, MZI_baseline_down = background_flattening(MZI_crop, lam=100*len(MZI_crop), baseline_up=True, baseline_down=True)

%matplotlib inline
fig, ax = plt.subplots(2,2,figsize=(13,4))
fig.tight_layout()

ax[0,0].plot(MZI_crop)
ax[0,0].plot(MZI_baseline_up, c='black')
ax[0,0].plot(MZI_baseline_down, c='black')
ax[0,0].grid(True)

ax[0,1].plot(MZI_normalized)
ax[0,1].grid(True)

idx_MZI_plot_i = 100000
idx_MZI_plot_f = 101000
ax[1,0].plot(MZI_crop[idx_MZI_plot_i:idx_MZI_plot_f])
ax[1,0].plot(MZI_baseline_up[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,0].plot(MZI_baseline_down[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,0].grid(True)

idx_MZI_plot_offset = 10000
idx_MZI_plot_i = idx_MZI_plot_i + idx_MZI_plot_offset
idx_MZI_plot_f = idx_MZI_plot_f + idx_MZI_plot_offset
ax[1,1].plot(MZI_crop[idx_MZI_plot_i:idx_MZI_plot_f])
ax[1,1].plot(MZI_baseline_up[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,1].plot(MZI_baseline_down[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,1].grid(True)

print(file_name)

In [ ]:
filt = False
if filt:
    MZI_filtered = savgol_filter(MZI_normalized, window_length=2, polyorder=1, mode='interp')
else:
    MZI_filtered = MZI_normalized

%matplotlib inline
fig, ax = plt.subplots(1,2,figsize=(13,2))
fig.tight_layout()

ax[0].plot(MZI_normalized, linewidth=3)
ax[0].plot(MZI_filtered, linewidth=1)
ax[0].set_ylabel("MZI Filt vs MZI Crop")
ax[0].grid(True)

idx_MZI_plot_i = 144370
idx_MZI_plot_f = 144470
ax[1].plot(MZI_normalized[idx_MZI_plot_i:idx_MZI_plot_f], linewidth=3)
ax[1].plot(MZI_filtered[idx_MZI_plot_i:idx_MZI_plot_f], linewidth=1)
ax[1].grid(True)

print(file_name)

In [ ]:
FSR = 137.0e6 # in HZ
detuning, data, MZI, idx_MZI_peaks = from_MZI_to_detuning(data_crop, MZI_filtered, FSR=FSR, distance=2, height=0.2, centralize=True)

%matplotlib inline
fig, ax = plt.subplots(2,1,figsize=(13,4))
fig.tight_layout()

ax[0].plot(detuning, MZI, linewidth=1)
ax[0].scatter(detuning[idx_MZI_peaks], MZI[idx_MZI_peaks], c='red', zorder=5)
ax[0].set_xlim(1e10,1.5e10)
ax[0].set_ylabel("MZI Norm.")
ax[0].set_xlabel("Detuning [units of FSR]")
ax[0].grid(True)

ax[1].plot(detuning, data, linewidth=3)
ax[1].set_xlim(detuning[0], detuning[-1])
ax[1].set_ylabel("Data")
ax[1].set_xlabel("Detuning [Hz]")
ax[1].grid(True)

print(file_name)

In [ ]:
data_normalized, data_baseline_up, data_baseline_down = background_flattening(data, lam=10**5*len(data), baseline_up=True, baseline_down=False)
data_normalized_peaks_height = 0.6
#idx_data_normalized_peaks = find_peaks(data_normalized, height=data_normalized_peaks_height, distance=1000)[0]
idx_data_normalized_peaks = find_peaks(1-data_normalized, height=data_normalized_peaks_height, distance=1000)[0]

%matplotlib inline
fig, ax = plt.subplots(1,2,figsize=(13,4))
fig.tight_layout()

ax[0].plot(data_normalized)
ax[0].plot([0, len(data_normalized)], [data_normalized_peaks_height, data_normalized_peaks_height], c="black", linestyle="dashed")
ax[0].scatter(idx_data_normalized_peaks, data_normalized[idx_data_normalized_peaks], c='red')
ax[0].grid(True)
counter=0
for i in idx_data_normalized_peaks:
    ax[0].annotate(str(counter) + " - " + str(i), xy=(i, data_normalized[i]), size=7, rotation=-90)
    counter += 1

ax[1].plot(data)
ax[1].plot(data_baseline_up, c='black')
ax[1].plot(data_baseline_down, c='black')
ax[1].scatter(idx_data_normalized_peaks, data[idx_data_normalized_peaks], c='red')
ax[1].grid(True)
counter=0
for i in idx_data_normalized_peaks:
    ax[1].annotate(str(counter) + " - " + str(i), xy=(i, data[i]), size=7, rotation=-90)
    counter += 1

print(file_name)

# Fitting Modes

In [ ]:
mode = 6
MZI_window_size = 500
MZI_plot_rescale_factor = 1/100
#-------

idx_mode = idx_data_peaks[mode]
closest_idx_MZI_peak = np.argmin(np.abs(idx_MZI_peaks - idx_mode))
idx_MZI_peaks_i = idx_MZI_peaks[closest_idx_MZI_peak - MZI_window_size]
idx_MZI_peaks_f = idx_MZI_peaks[closest_idx_MZI_peak + MZI_window_size]

X_mode = (detuning - detuning[idx_mode])[idx_MZI_peaks_i:(idx_MZI_peaks_f+1)]
Y_mode = data[idx_MZI_peaks_i:(idx_MZI_peaks_f+1)]
MZI_mode = MZI[idx_MZI_peaks_i:(idx_MZI_peaks_f+1)]*MZI_plot_rescale_factor
idx_MZI_peaks_mode = idx_MZI_peaks[(closest_idx_MZI_peak - MZI_window_size):(closest_idx_MZI_peak + MZI_window_size)] - idx_MZI_peaks_i 

caique_plot(X_mode, Y_mode, MZI=MZI_mode, idx_MZI_peaks=idx_MZI_peaks_mode)

## Single Optical Mode

In [ ]:
#optical_mode(Δ, Δ0, κ, κe, V0)
model = lmfit.Model(optical_mode)
params = model.make_params(Δ0={'value': 0.0e9, 'vary': True, 'min':-5.0e9, 'max':5.0e9},
                            κ={'value': 0.35e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           κe={'value': 0.1e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           V0={'value': 0.01, 'vary': True, 'min':0.0, 'max':np.inf})
result = model.fit(Y_mode, params, Δ=X_mode, method='leastsq')

caique_plot(X_mode, Y_mode, MZI=MZI_mode, idx_MZI_peaks=idx_MZI_peaks_mode, result=result)

## Splitted Optical Mode

In [ ]:
#splitted_optical_mode(Δ, Δ0, κ, κe, V0, J)
model = lmfit.Model(splitted_optical_mode)
params = model.make_params(Δ0={'value': -0.2e9, 'vary': True, 'min':-5.0e9, 'max':5.0e9},
                            κ={'value': 0.4e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           κe={'value': 0.10e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           V0={'value': 1.2, 'vary': True, 'min':0.0, 'max':np.inf},
                            J={'value': 0.2e9, 'vary': True, 'min':-10.0e9, 'max':10.0e9})
result = model.fit(Y_mode, params, Δ=X_mode, method='leastsq')

caique_plot(X_mode, Y_mode, MZI=MZI_mode, idx_MZI_peaks=idx_MZI_peaks_mode, result=result)

## Single Optical Mode with Fabry Perot background

In [ ]:
#optical_mode_with_fabry_perot(Δ, Δ0, κ, κe, V0, R1, R2, δ, δΔ)
model = lmfit.Model(optical_mode_with_fabry_perot)
model.set_param_hint('R2', expr='R1')
params = model.make_params(Δ0={'value': -0.05e9, 'vary': True, 'min':-5.0e9, 'max':5.0e9},
                            κ={'value': 2.0e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           κe={'value': 0.5e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           V0={'value': 0.015, 'vary': True, 'min':0.0, 'max':np.inf},
                           R1={'value': 0.01, 'vary': True, 'min':0.0, 'max':0.1},
                           #R2={'value': 0.01, 'vary': True, 'min':0.0, 'max':0.03},
                            δ={'value': 1.1e-9, 'vary': True, 'min':0.0e-9, 'max':10e-9},
                           δΔ={'value': 2e9, 'vary': True, 'min':-10.0e9, 'max':10.0e9})
result = model.fit(Y_mode, params, Δ=X_mode, method='leastsq')

caique_plot(X_mode, Y_mode, MZI=MZI_mode, idx_MZI_peaks=idx_MZI_peaks_mode, result=result)

## Splitted Optical Mode with Fabry Perot background

In [ ]:
#splitted_optical_mode_with_fabry_perot(Δ, Δ0, κ, κe, V0, J, R1, R2, δin, δΔin, δout, δΔout)
model = lmfit.Model(splitted_optical_mode_with_fabry_perot)
model.set_param_hint('R2', expr='R1')
params = model.make_params(Δ0={'value': -0.02e9, 'vary': True, 'min':-3.0e9, 'max':3.0e9},
                            κ={'value': 0.4e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           κe={'value': 0.1e9, 'vary': True, 'min':0.0, 'max':np.inf},
                           V0={'value': 1.1407561, 'vary': True, 'min':0.0, 'max':np.inf},
                            J={'value': 0.3e9, 'vary': True, 'min':-5e9, 'max':5e9},
                           R1={'value': 0.02, 'vary': True, 'min':0.0, 'max':0.2},
                           #R2={'value': 0.01, 'vary': True, 'min':0.0, 'max':0.2},
                          δin={'value': 0.2e-9, 'vary': True, 'min':-10e-9, 'max':10e-9},
                         δΔin={'value': 1.7e9, 'vary': True, 'min':-10.0e9, 'max':10.0e9},
                         δout={'value': 0.2e-9, 'vary': True, 'min':-10e-9, 'max':10e-9},
                        δΔout={'value': 1.7e9, 'vary': True, 'min':-10.0e9, 'max':10.0e9})
result = model.fit(Y_mode, params, Δ=X_mode, method='leastsq')

caique_plot(X_mode, Y_mode, MZI=MZI_mode, idx_MZI_peaks=idx_MZI_peaks_mode, result=result)

# Dispersion Analysis

In [ ]:
mode_to_center = 7
X_disp = (np.arange(len(idx_data_peaks)) - mode_to_center)
Y_disp = (detuning[idx_data_peaks] - detuning[idx_data_peaks[mode_to_center]])
#----------

model = lmfit.Model(freq_disp)
params = model.make_params(D0={'value': 5.0e12, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D1={'value': 170.0e9, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D2={'value': -50.0e6, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D3={'value': 1.0e7, 'vary': True, 'min':-np.inf, 'max':np.inf})
result = model.fit(Y_disp, params, u=X_disp, method='leastsq')

ans = result.params.valuesdict()

fig, ax = plt.subplots(figsize=(13,4))
fig.tight_layout()
ax.scatter(X_disp, (Y_disp - ans['D0'] - X_disp*ans['D1']), color='black', linewidth=3)
ax.plot(X_disp, freq_disp(X_disp, 0, 0, ans['D2'], ans['D3']), color='red', linewidth=2, linestyle='dashed')
ax.grid(True)
print(file_name)

caique_plot(X_disp, Y_disp, plottype='scatter', annotate=False, result=result)

In [13]:
fig.savefig(selected_files[idx_file].split(".")[-2].split(".")[0]+"spectrumTE.pdf", dpi=600)

In [ ]:
X_disp_new = (detuning[idx_data_peaks] - detuning[idx_data_peaks[mode_to_center]])

fig, ax = plt.subplots(figsize=(13,4))
fig.tight_layout()
ax.scatter(X_disp_new, (Y_disp - ans['D0'] - X_disp*ans['D1']), color='black', linewidth=3)
ax.plot(X_disp_new, freq_disp(X_disp, 0, 0, ans['D2'], ans['D3']), color='red', linewidth=2, linestyle='dashed')
ax.grid(True)
ax.set_ylim(-2.1e9,0.1e9)
ax.set_xlim(-1e12,1e12)
print(file_name)